오전
캐글 - 타이타닉 데이터 EDA
생존/사망 예측 모델 구현
규칙 기반 모델(IF 조건 THEN 사망/생존)
모델 : if 무조건 then 사망 -> 예측 => 제출

오후
모델링 및 제출(10번 제한) : 1시~5시20분 
~~최적의 모델링 설계 방법(수업) : 5시20분~5시50분~~
제출 평가 : 5시20분~5시50분
목표 : 오전에 제출한 모델의 예측 결과 랭킹을 개선
카페 제출 : 최종 랭킹 캡쳐 화면

이번 제출에서는 머신러닝, 딥러닝없이 제출하는 것으로 진행
치팅 금지
머신러닝 같은 것을 이용하여 정확도 0.9를 넘기기가 굉장히 어려우므로 타이타닉에서 1은 치팅이라고 볼 수 있음
knn이 0.80622정도 했다고 하심

가장 점수 높은 3명은 우수학생 선발때 가산점으로 반영할 것임

In [ ]:
# 캐글 competitions에서 입상하면 연락이 온다고 함

In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import *

In [162]:
train_df = pd.read_csv('titanic/train.csv')
train_df.info()
train_df.describe() # nan값이 있는 데이터는 age, Cabin, Embarked 3가지 열임

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# train_df -> 데이터의 패턴 발견(EDA)-> 패턴식을 정의(모델) -> 모델에 test_df 입력 -> 생존/사망 예측
# => 예측 결과를 csv 파일로 저장 => 캐글 제출

# 규칙 기반 패턴 예시
# 1)
# if 성별이 남성 then 사망
# if 성별이 여성 then 생존

# 2)
# if 나이가 20세 미만 then 생존
# if 나이가 60세 이상 then 생존
# if 나이가 20~60세 then 사망

# ...

In [163]:
# 데이터중 필요없는 것으로 예상되는 컬럼 제거 = 'PassengerId','Ticket'
# 결측치가 많은 Cabin열 제거
train_df.drop(['PassengerId','Ticket','Cabin'], axis=1, inplace=True)
train_df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C


# Sex열은 원핫인코딩

In [164]:
ohe_sex = OneHotEncoder().fit_transform(train_df[['Sex']]).toarray()
sex = pd.DataFrame(ohe_sex, columns=['female', 'male'])
sex

,female,male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0
...,...,...
886,0.0,1.0
887,1.0,0.0
888,1.0,0.0
889,0.0,1.0


In [165]:
train_df.drop('Sex', axis=1, inplace=True)

In [166]:
train_df = pd.concat([train_df, sex], axis=1)
train_df

,Survived,Pclass,Name,Age,SibSp,Parch,Fare,Embarked,female,male
0,0,3,"Braund, Mr. Owen Harris",22.0,1,0,7.2500,S,0.0,1.0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,71.2833,C,1.0,0.0
2,1,3,"Heikkinen, Miss. Laina",26.0,0,0,7.9250,S,1.0,0.0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,53.1000,S,1.0,0.0
4,0,3,"Allen, Mr. William Henry",35.0,0,0,8.0500,S,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",27.0,0,0,13.0000,S,0.0,1.0
887,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,30.0000,S,1.0,0.0
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,23.4500,S,1.0,0.0
889,1,1,"Behr, Mr. Karl Howell",26.0,0,0,30.0000,C,0.0,1.0


In [167]:
# 사망자 통계
train_df[train_df['Survived'] == 0].describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male
count,549.0,549.000000,424.000000,549.000000,549.000000,549.000000,549.000000,549.000000
mean,0.0,2.531876,30.626179,0.553734,0.329690,22.117887,0.147541,0.852459
std,0.0,0.735805,14.172110,1.288399,0.823166,31.388207,0.354968,0.354968
min,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,2.000000,21.000000,0.000000,0.000000,7.854200,0.000000,1.000000
50%,0.0,3.000000,28.000000,0.000000,0.000000,10.500000,0.000000,1.000000
75%,0.0,3.000000,39.000000,1.000000,0.000000,26.000000,0.000000,1.000000
max,0.0,3.000000,74.000000,8.000000,6.000000,263.000000,1.000000,1.000000


In [168]:
# 생존자 통계
train_df[train_df['Survived'] == 1].describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male
count,342.0,342.000000,290.000000,342.000000,342.000000,342.000000,342.000000,342.000000
mean,1.0,1.950292,28.343690,0.473684,0.464912,48.395408,0.681287,0.318713
std,0.0,0.863321,14.950952,0.708688,0.771712,66.596998,0.466660,0.466660
min,1.0,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,1.000000,19.000000,0.000000,0.000000,12.475000,0.000000,0.000000
50%,1.0,2.000000,28.000000,0.000000,0.000000,26.000000,1.000000,0.000000
75%,1.0,3.000000,36.000000,1.000000,1.000000,57.000000,1.000000,1.000000
max,1.0,3.000000,80.000000,4.000000,5.000000,512.329200,1.000000,1.000000


현재 가설로는 여성이 생존율이 더 높음

# 호칭 분류

In [212]:
# 정규식을 이용한 이름 분류
import re
def find_name_title(name):
    return re.search('[A-z]+\.', name).group()[:-1]
train_df['name_title'] = train_df.Name.apply(find_name_title)
train_df.drop('Name', axis=1, inplace=True) # 이름에서 호칭을 분리했으므로 삭제
train_df

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,female,male,name_title
0,0,3,22.0,1,0,7.2500,S,0.0,1.0,Mr
1,1,1,38.0,1,0,71.2833,C,1.0,0.0,Mrs
2,1,3,26.0,0,0,7.9250,S,1.0,0.0,Miss
3,1,1,35.0,1,0,53.1000,S,1.0,0.0,Mrs
4,0,3,35.0,0,0,8.0500,S,0.0,1.0,Mr
...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,S,0.0,1.0,Rev
887,1,1,19.0,0,0,30.0000,S,1.0,0.0,Miss
888,0,3,NaN,1,2,23.4500,S,1.0,0.0,Miss
889,1,1,26.0,0,0,30.0000,C,0.0,1.0,Mr


In [218]:
train_df.name_title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: name_title, dtype: int64

In [221]:
train_df[train_df.Survived == 1].name_title.value_counts()

Miss        127
Mrs          99
Mr           81
Master       23
Dr            3
Mlle          2
Mme           1
Ms            1
Major         1
Lady          1
Sir           1
Col           1
Countess      1
Name: name_title, dtype: int64

Mr : 남성
Mrs : 결혼한 여성
Miss : 미혼 여성
Master : 남성?

미혼 여성보다 혼인 여성이 더 생존율은 높음
master는 생존율이 절반정도 되는 한편 Mr는 생존율이 20%에 이를정도로 낮음

# 나이대 묶기

In [326]:
def age_alive(age):
    if(age < 10):
        return 0
    elif(age < 20):
        return 1
    elif(age < 30):
        return 2
    elif(age < 40):
        return 3
    elif(age < 50):
        return 4
    elif(age < 60):
        return 5
    elif(age < 70):
        return 6
    elif(age < 80):
        return 7
    elif(age < 90):
        return 8
train_df['age_range'] = train_df.Age.apply(age_alive)
train_df.drop('Age', axis=1, inplace=True) # 나이대로 분류했으므로 상세 나이는 삭제

In [327]:
train_df

,Survived,Pclass,SibSp,Parch,Fare,Embarked,female,male,name_title,age_range
0,0,3,1,0,7.2500,S,0.0,1.0,Mr,2.0
1,1,1,1,0,71.2833,C,1.0,0.0,Mrs,3.0
2,1,3,0,0,7.9250,S,1.0,0.0,Miss,2.0
3,1,1,1,0,53.1000,S,1.0,0.0,Mrs,3.0
4,0,3,0,0,8.0500,S,0.0,1.0,Mr,3.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,S,0.0,1.0,Rev,2.0
887,1,1,0,0,30.0000,S,1.0,0.0,Miss,1.0
888,0,3,1,2,23.4500,S,1.0,0.0,Miss,NaN
889,1,1,0,0,30.0000,C,0.0,1.0,Mr,2.0


# 나이대 추정

In [434]:
train_df[['Pclass', 'Survived']].value_counts()
# pclass는 3>1>2순으로 사망률이 높음(3이 제일 사망률 높음)

Pclass  Survived
3       0           372
1       1           136
3       1           119
2       0            97
        1            87
1       0            80
dtype: int64

In [329]:
train_df[~train_df.age_range.isnull()][['Pclass','name_title','age_range']].value_counts().iloc[:25]

Pclass  name_title  age_range
3       Mr          2.0          102
                    3.0           53
                    1.0           39
2       Mr          3.0           31
3       Mr          4.0           25
2       Mr          2.0           25
3       Master      0.0           21
        Miss        2.0           21
1       Mr          3.0           21
3       Miss        1.0           21
                    0.0           21
1       Mr          4.0           20
                    2.0           18
2       Mrs         2.0           16
1       Miss        3.0           16
        Mr          5.0           12
3       Mrs         3.0           11
1       Miss        2.0           11
3       Mrs         2.0           10
1       Mrs         3.0           10
2       Mr          1.0           10
        Mrs         3.0           10
                    4.0            9
        Master      0.0            9
1       Miss        1.0            9
dtype: int64

In [330]:
train_df[train_df.age_range.isnull()][['Pclass','name_title','Survived']].value_counts()

Pclass  name_title  Survived
3       Mr          0           83
        Miss        1           19
1       Mr          0           15
3       Miss        0           14
1       Mrs         1            8
2       Mr          0            7
3       Mr          1            7
        Mrs         1            6
1       Mr          1            5
3       Mrs         0            3
2       Miss        1            2
        Mr          1            2
3       Master      0            2
                    1            2
1       Dr          0            1
        Miss        1            1
dtype: int64

In [323]:
# pclass가 3이며 호칭이 Mr이고 사망자는 20대일 가능성이 높아보임
# (1은 가격때문에 비싸서 잘 이용을 못하고 3은 저렴한 좌석일듯)

train_df.loc[(train_df.Pclass == 3) & (train_df.name_title == 'Mr') & (train_df.Survived == 0) & (train_df.age_range.isnull()), 'age_range'] = 2

In [346]:
# Miss의 연령대별 생존수
train_df[train_df.name_title == 'Miss'][['age_range', 'Survived']].value_counts().sort_values(ascending=False)

age_range  Survived
2.0        1           27
1.0        1           26
3.0        1           23
0.0        1           19
2.0        0           13
0.0        0           11
1.0        0           10
3.0        0            5
5.0        1            5
4.0        1            4
           0            1
5.0        0            1
6.0        1            1
dtype: int64

In [351]:
train_df[(train_df['age_range'] >= 0) & (train_df['age_range'] <= 3) & (train_df.Survived == 1)].describe()

,Survived,Pclass,SibSp,Parch,Fare,female,male,age_range
count,229.0,229.000000,229.000000,229.000000,229.000000,229.000000,229.000000,229.00000
mean,1.0,2.008734,0.506550,0.585153,49.054313,0.676856,0.323144,1.80786
std,0.0,0.842882,0.770168,0.847052,73.592513,0.468702,0.468702,1.06282
min,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.0,1.000000,0.000000,0.000000,12.350000,0.000000,0.000000,1.00000
50%,1.0,2.000000,0.000000,0.000000,26.000000,1.000000,0.000000,2.00000
75%,1.0,3.000000,1.000000,1.000000,56.495800,1.000000,1.000000,3.00000
max,1.0,3.000000,4.000000,5.000000,512.329200,1.000000,1.000000,3.00000


In [352]:
train_df[(train_df['age_range'] >= 0) & (train_df['age_range'] <= 3) & (train_df.Survived == 0)].describe()

,Survived,Pclass,SibSp,Parch,Fare,female,male,age_range
count,322.0,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000
mean,0.0,2.630435,0.611801,0.344720,20.885945,0.161491,0.838509,1.953416
std,0.0,0.624080,1.155341,0.794718,30.354616,0.368556,0.368556,0.883037
min,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,2.000000,0.000000,0.000000,7.895800,0.000000,1.000000,1.000000
50%,0.0,3.000000,0.000000,0.000000,10.481250,0.000000,1.000000,2.000000
75%,0.0,3.000000,1.000000,0.000000,23.631250,0.000000,1.000000,3.000000
max,0.0,3.000000,5.000000,5.000000,263.000000,1.000000,1.000000,3.000000


생존한 Miss의 경우 Fare의 수치가 상당히 컸음. 또한 Pclass가 보통 2정도에 몰려 있음

In [120]:
train_df[train_df['Survived'] == 0].Embarked.value_counts()

S    427
C     75
Q     47
Name: Embarked, dtype: int64

In [121]:
train_df[train_df['Survived'] == 1].Embarked.value_counts() #  C값이 더 생존율이 높음;

S    217
C     93
Q     30
Name: Embarked, dtype: int64

위 내용을 보았을 때 Pclass,sibsp는 낮을수록, Fare는 높을수록 생존율이 높은 경향을 보임

In [101]:
train_df.groupby('Sex')['Survived'].sum()# 여성 생존율이 더 높음

Sex
female    233
male      109
Name: Survived, dtype: int64

In [102]:
# 생존자 정보 분석(pclass)
pclass = train_df[train_df['Survived'] == 1].Pclass
pclass.value_counts() # 1등급이 더 많이 생존함.

1    136
3    119
2     87
Name: Pclass, dtype: int64

In [357]:
train_df.corr()

C:\Users\fkfma\AppData\Local\Temp\ipykernel_15456\1233823885.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_df.corr()


,Survived,Pclass,SibSp,Parch,Fare,female,male,age_range
Survived,1.000000,-0.338481,-0.035322,0.081629,0.257307,0.543351,-0.543351,-0.073029
Pclass,-0.338481,1.000000,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.352894
SibSp,-0.035322,0.083081,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.300487
Parch,0.081629,0.018443,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.189679
Fare,0.257307,-0.549500,0.159651,0.216225,1.000000,0.182333,-0.182333,0.089421
female,0.543351,-0.131900,0.114631,0.245489,0.182333,1.000000,-1.000000,-0.093465
male,-0.543351,0.131900,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,0.093465
age_range,-0.073029,-0.352894,-0.300487,-0.189679,0.089421,-0.093465,0.093465,1.000000


상관관계로는 성별이 생존과 가장 큰 연관성을 보임. 이외는 Fare(운임비용)이 상관계수가 크지만 큰 의미를 갖지는 못할 것 같음

In [362]:
train_df[['Pclass','Survived']].value_counts()

Pclass  Survived
3       0           372
1       1           136
3       1           119
2       0            97
        1            87
1       0            80
dtype: int64

In [382]:
# pclass와 age_range도 관련이 있으므로 확인
train_df[train_df.female == 1][['Pclass','age_range','Survived']].value_counts()[:15]
# pclass가 1에 가까울 수록 생존율은 높으나 2일 경우는 2~30대이며 생존율이 pclass가 1일때보다 평균적으로 높음

Pclass  age_range  Survived
1       3.0        1           27
2       2.0        1           22
        3.0        1           16
3       2.0        0           16
1       2.0        1           15
3       2.0        1           15
        1.0        1           13
1       4.0        1           13
        1.0        1           13
3       0.0        1           11
        1.0        0           11
1       5.0        1           11
3       0.0        0           10
        3.0        0            9
2       4.0        1            9
dtype: int64

여기까지의 내용으로 봤을 때, 
1. 여성이 생존율 높음
2. 남성은 생존율이 낮음
3. 여성은 특히 Miss 이면서 나이대가 30대 이하인 경우 생존율이 상승함
4. 추가로 embark는 C일 경우 생존율이 높았음(이유는 전혀 모르겠음)
5. Pclass는 2일 경우 생존율은 가장 높았음. 이건 약간 특이 케이스로 보이며 실제로는 1이 제일 좋은 좌석일 듯
현재까지의 정보로 한번 데이터를 제출해보기로..

In [368]:
test_df = pd.read_csv('titanic/test.csv')
test_df['res'] = 0 # 임의의 데이터
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,res
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0


In [418]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        107 non-null    object 
 10  Embarked     418 non-null    object 
 11  res          418 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [513]:
gender_submission = pd.read_csv('titanic/gender_submission.csv')
test_df = pd.read_csv('titanic/test.csv')

In [514]:
test_df['name_title'] = test_df.Name.apply(find_name_title)
test_df.drop('Name', axis=1, inplace=True) # 이름에서 호칭을 분리했으므로 삭제
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,Mrs
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
414,1306,1,female,39.0,0,0,PC 17758,108.9000,C105,C,Dona
415,1307,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr
416,1308,3,male,NaN,0,0,359309,8.0500,NaN,S,Mr


In [515]:
test_df['family_size'] = test_df.SibSp + test_df.Parch
test_df['res']= 0
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title,family_size,res
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,Mr,0,0
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,Mrs,1,0
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,Mr,0,0
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,Mr,0,0
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr,0,0
414,1306,1,female,39.0,0,0,PC 17758,108.9000,C105,C,Dona,0,0
415,1307,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,0,0
416,1308,3,male,NaN,0,0,359309,8.0500,NaN,S,Mr,0,0


In [516]:
test_df.loc[(test_df.Pclass == 1) & (test_df.Age < 40) & (test_df.Sex == 'female'), 'res'] = 1
test_df.loc[(test_df.Age < 40) & (test_df.Sex == 'female') & (test_df.Fare > 50), 'res'] = 1
test_df.loc[(test_df.Pclass == 2) & (test_df.Age >= 20) & (test_df.Age < 40) & (test_df.Sex == 'female')] = 1
# 2회차 코드 추가
test_df.loc[(test_df.Embarked == 'C') & (test_df.Sex == 'female'), 'res'] = 1
# 3회차 코드 추가
test_df['age_range'] = test_df.Age.apply(age_alive)
test_df.drop('Age', axis=1, inplace=True)

test_df.loc[(test_df.name_title == 'Miss') & (test_df.Sex == 'female') & (test_df.age_range <= 3), 'res'] = 1
# 4회차 코드 수정
test_df.loc[(test_df.name_title == 'Miss') & (test_df.Sex == 'female'), 'res'] = 1
# 5회차 코드 추가
test_df.loc[(test_df.family_size >= 1) & (test_df.family_size <= 3), 'res'] = 1
test_df

,PassengerId,Pclass,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title,family_size,res,age_range
0,892,3,male,0,0,330911,7.8292,NaN,Q,Mr,0,0,3.0
1,893,3,female,1,0,363272,7.0000,NaN,S,Mrs,1,1,4.0
2,894,2,male,0,0,240276,9.6875,NaN,Q,Mr,0,0,6.0
3,895,3,male,0,0,315154,8.6625,NaN,S,Mr,0,0,2.0
4,896,3,female,1,1,3101298,12.2875,NaN,S,Mrs,2,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,0,0,A.5. 3236,8.0500,NaN,S,Mr,0,0,NaN
414,1306,1,female,0,0,PC 17758,108.9000,C105,C,Dona,0,1,3.0
415,1307,3,male,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,0,0,3.0
416,1308,3,male,0,0,359309,8.0500,NaN,S,Mr,0,0,NaN


In [503]:
gender_submission = pd.read_csv('titanic/gender_submission.csv')
gender_submission.Survived = test_df.res

In [387]:
gender_submission.to_csv('my_submission240610_1.csv', index=False)

# 여자관련 데이터만 처리해둔 상태임.
# Score: 0.70334

In [400]:
gender_submission.to_csv('my_submission240610_2.csv', index=False)

# 여자관련 데이터만 처리해둠
# 0.75598

In [475]:
gender_submission.to_csv('my_submission240610_3.csv', index=False)

# 여자관련 데이터만 처리해둠
# 0.76555 상승폭이 작음...

In [478]:
gender_submission.to_csv('my_submission240610_4.csv', index=False)

# 여자관련 데이터만 처리해둠
# 0.76555 이전과 동일한 점수가 나옴(아마 변화가 없던 모양임)

In [517]:
gender_submission.to_csv('my_submission240610_5.csv', index=False)

# 가족수까지 맞춰서 처리
# 0.76555 ...?

In [502]:
test_df

,PassengerId,Pclass,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,name_title,family_size,res,age_range
0,892,3,male,0,0,330911,7.8292,NaN,Q,Mr,0,NaN,3.0
1,893,3,female,1,0,363272,7.0000,NaN,S,Mrs,1,1.0,4.0
2,894,2,male,0,0,240276,9.6875,NaN,Q,Mr,0,NaN,6.0
3,895,3,male,0,0,315154,8.6625,NaN,S,Mr,0,NaN,2.0
4,896,3,female,1,1,3101298,12.2875,NaN,S,Mrs,2,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,0,0,A.5. 3236,8.0500,NaN,S,Mr,0,NaN,NaN
414,1306,1,female,0,0,PC 17758,108.9000,C105,C,Dona,0,1.0,3.0
415,1307,3,male,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,0,NaN,3.0
416,1308,3,male,0,0,359309,8.0500,NaN,S,Mr,0,NaN,NaN


In [388]:
train_df

,Survived,Pclass,SibSp,Parch,Fare,Embarked,female,male,name_title,age_range
0,0,3,1,0,7.2500,S,0.0,1.0,Mr,2.0
1,1,1,1,0,71.2833,C,1.0,0.0,Mrs,3.0
2,1,3,0,0,7.9250,S,1.0,0.0,Miss,2.0
3,1,1,1,0,53.1000,S,1.0,0.0,Mrs,3.0
4,0,3,0,0,8.0500,S,0.0,1.0,Mr,3.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,S,0.0,1.0,Rev,2.0
887,1,1,0,0,30.0000,S,1.0,0.0,Miss,1.0
888,0,3,1,2,23.4500,S,1.0,0.0,Miss,NaN
889,1,1,0,0,30.0000,C,0.0,1.0,Mr,2.0


In [398]:
train_df.groupby(['male','female'])['Embarked', 'Survived'].value_counts()

C:\Users\fkfma\AppData\Local\Temp\ipykernel_15456\3140959012.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  train_df.groupby(['male','female'])['Embarked', 'Survived'].value_counts()


male  female  Embarked  Survived
0.0   1.0     S         1           140
              C         1            64
              S         0            63
              Q         1            27
              C         0             9
              Q         0             9
1.0   0.0     S         0           364
                        1            77
              C         0            66
              Q         0            38
              C         1            29
              Q         1             3
dtype: int64

?? 여성이 embarked가 C면 압도적으로 생존율이 높아짐;;

In [395]:
test_df.groupby('Sex')[['Embarked']].value_counts()

Sex     Embarked
1       1            18
female  S            73
        C            37
        Q            24
male    S           182
        C            62
        Q            22
dtype: int64

In [480]:
man_alive = train_df[(train_df.male == 1) & (train_df.Survived == 1)].drop(['Survived','female','male', 'name_title'], axis=1)
man_dead = train_df[(train_df.male == 1) & (train_df.Survived == 0)].drop(['Survived','female','male', 'name_title'], axis=1)
man_alive.describe()

,Pclass,SibSp,Parch,Fare,age_range
count,109.000000,109.000000,109.000000,109.000000,93.000000
mean,2.018349,0.385321,0.357798,40.821484,2.301075
std,0.922774,0.636952,0.645826,71.355967,1.647033
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,9.500000,1.000000
50%,2.000000,0.000000,0.000000,26.287500,2.000000
75%,3.000000,1.000000,1.000000,39.000000,3.000000
max,3.000000,4.000000,2.000000,512.329200,8.000000


In [417]:
man_dead.describe()

,Pclass,SibSp,Parch,Fare,age_range
count,468.000000,468.000000,468.000000,468.000000,360.000000
mean,2.476496,0.440171,0.207265,21.960993,2.716667
std,0.761411,1.138507,0.601382,32.409799,1.467475
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,7.795800,2.000000
50%,3.000000,0.000000,0.000000,9.416650,2.000000
75%,3.000000,0.000000,0.000000,26.000000,3.250000
max,3.000000,8.000000,5.000000,263.000000,7.000000


sibSp가 낮을수록 생존율이 높은듯함

남성의 생존율을 분석하기는 난이도가 높음..

In [440]:
# 다시 연령을 분석해보기로 = null값이 있는 데이터가 상당히 많으므로 이를 보완하더라도 정확도가 다소 높아질 수 있음
test_df[test_df.Sex == 'female']

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S
...,...,...,...,...,...,...,...,...,...,...,...
409,1301,3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S
410,1302,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.7500,NaN,Q
411,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q
412,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S


In [461]:
train_df

,Survived,Pclass,SibSp,Parch,Fare,Embarked,female,male,name_title,age_range
0,0,3,1,0,7.2500,S,0.0,1.0,Mr,2.0
1,1,1,1,0,71.2833,C,1.0,0.0,Mrs,3.0
2,1,3,0,0,7.9250,S,1.0,0.0,Miss,2.0
3,1,1,1,0,53.1000,S,1.0,0.0,Mrs,3.0
4,0,3,0,0,8.0500,S,0.0,1.0,Mr,3.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,S,0.0,1.0,Rev,2.0
887,1,1,0,0,30.0000,S,1.0,0.0,Miss,1.0
888,0,3,1,2,23.4500,S,1.0,0.0,Miss,NaN
889,1,1,0,0,30.0000,C,0.0,1.0,Mr,2.0


In [462]:
train_df[(train_df.name_title == 'Miss') & train_df.Survived].age_range.value_counts()

2.0    27
1.0    26
3.0    23
0.0    19
5.0     5
4.0     4
6.0     1
Name: age_range, dtype: int64

In [463]:
train_df[(train_df.name_title == 'Miss') & (train_df.Survived == 0)].age_range.value_counts()

2.0    13
0.0    11
1.0    10
3.0     5
5.0     1
4.0     1
Name: age_range, dtype: int64

미혼자가 더 생존율이 높음

In [497]:
gender_submission = pd.read_csv('titanic/gender_submission.csv')
test_df = pd.read_csv('titanic/test.csv')
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# 가족 체크

In [486]:
train_df['family_size'] = train_df.SibSp + train_df.Parch
train_df

,Survived,Pclass,SibSp,Parch,Fare,Embarked,female,male,name_title,age_range,family_size
0,0,3,1,0,7.2500,S,0.0,1.0,Mr,2.0,1
1,1,1,1,0,71.2833,C,1.0,0.0,Mrs,3.0,1
2,1,3,0,0,7.9250,S,1.0,0.0,Miss,2.0,0
3,1,1,1,0,53.1000,S,1.0,0.0,Mrs,3.0,1
4,0,3,0,0,8.0500,S,0.0,1.0,Mr,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.0000,S,0.0,1.0,Rev,2.0,0
887,1,1,0,0,30.0000,S,1.0,0.0,Miss,1.0,0
888,0,3,1,2,23.4500,S,1.0,0.0,Miss,NaN,3
889,1,1,0,0,30.0000,C,0.0,1.0,Mr,2.0,0


In [493]:
train_df[train_df.Survived == 1][['Survived','family_size']].value_counts()

Survived  family_size
1         0              163
          1               89
          2               59
          3               21
          6                4
          4                3
          5                3
dtype: int64

In [494]:
train_df[train_df.Survived == 0][['Survived','family_size']].value_counts()

Survived  family_size
0         0              374
          1               72
          2               43
          5               19
          4               12
          3                8
          6                8
          10               7
          7                6
dtype: int64

? 가족수가 1,2,3 이면 생존율이 더 높음.. 아마 가족수가 많으면 휘말리고, 가족수가 적으면 다같이 탈출할 가능성이 높은 것 같음

In [48]:
# 결측값 처리? 제거, 기본값

# gender_submission의 Survived열 값을 모두 사망(0)으로 저장하시오.
gender_submission.Survived = 0
gender_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [50]:
# 제출용 파일 생성
gender_submission.to_csv("my_submission.csv", index=False)

In [51]:
pd.read_csv('my_submission.csv') # 15589 등으로 시작함

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [ ]:
# Mrs같은 것을 가지고(호칭) 확률이 나뉘어질 수도 있음

In [53]:
train_df.corr().Survived

C:\Users\fkfma\AppData\Local\Temp\ipykernel_15456\829844629.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_df.corr().Survived


PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Name: Survived, dtype: float64